In [108]:
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Pretty display for notebooks
%matplotlib inline


In [109]:
#Parameters
data_directory = "/Fergus/Code/Data"
file1 = "/Airbnb/Dublin/summary_listings.csv"
file2 = "/Airbnb/Dublin/listings.csv"
file3 = "/Airbnb/Dublin/reviews.csv/reviews.csv"
name_of_dependent_variable = 'income'
skewed = ['capital-gain', 'capital-loss']

numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

In [110]:
#Open data file
data_summary = pd.read_csv(data_directory + file1)
data_extended = pd.read_csv(data_directory + file2)
data_reviews = pd.read_csv(data_directory + file3)

C:\Users\Fergus\.conda\envs\uci_env\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [111]:
#EXPLORATION
#Display the first few records
display(data_summary.head(n=3))

data_summary.shape

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,14842,"Best location Portobello entire house, can sle...",58344,Hilary,NaN,Dublin City,53.331057,-6.269163,Entire home/apt,119,20,42,2019-01-02,0.46,1,5
1,44077,cosy comfortable Irish cottage twin,193005,Teresa,NaN,Dn Laoghaire-Rathdown,53.289950,-6.258126,Private room,50,3,182,2018-12-09,1.91,3,309
2,68483,Unique Central Dublin River-view,339817,John,NaN,Dublin City,53.345013,-6.265524,Entire home/apt,88,3,99,2018-11-06,1.04,1,84


(9520, 16)

In [5]:
print(data_extended.columns.tolist())

['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'experiences_offered', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'street', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 'smart_location', 'country_code', 'country', 'latitude', 'longitude', 'is_location_exact', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet', 'price', 'weekly_price', 'monthly_price', '

In [112]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
data_extended.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,14842,https://www.airbnb.com/rooms/14842,20190122134607,2019-01-22,"Best location Portobello entire house, can sle...","Beautiful entire house, newly revamped, with s...","Gorgeous, newly revamped central Dublin city e...","Beautiful entire house, newly revamped, with s...",none,Portobello is an exceptional place to live - v...,NaN,"There is great public transport - buses, Luas,...",Guests get the whole house to themselves.,I love meeting guests and I give you extensive...,"This house is a charming space, just completel...",NaN,NaN,https://a0.muscache.com/im/pictures/106656330/...,NaN,58344,https://www.airbnb.com/users/show/58344,Hilary,2009-11-29,"Dublin, County Dublin, Ireland",An easy-going and friendly Irish woman. Works ...,within an hour,100%,NaN,t,https://a0.muscache.com/im/users/58344/profile...,https://a0.muscache.com/im/users/58344/profile...,NaN,1,1,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,"Dublin, Dublin City, Ireland",Portobello,Dublin City,NaN,Dublin,Dublin City,Dublin 8,Dublin,"Dublin, Ireland",IE,Ireland,53.331057,-6.269163,f,House,Entire home/apt,3,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,Heating,""Family/kid ...",0.0,$119.00,NaN,NaN,$500.00,$60.00,2,$30.00,20,31,20,20,31,31,20.0,31.0,3 days ago,t,2,2,5,5,2019-01-22,42,16,2011-07-11,2019-01-02,95.0,10.0,9.0,10.0,10.0,10.0,9.0,f,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,1,0,0,0.46
1,44077,https://www.airbnb.com/rooms/44077,20190122134607,2019-01-22,cosy comfortable Irish cottage twin,Our house was built in 1937 when there was ple...,"Hi, My name is Teresa and we have 3 rooms ava...",Our house was built in 1937 when there was ple...,none,I like our neighbourhood as there is no shorta...,Dundrum was always known as a Spa Town and the...,There is a bus stop 2 mins up the road from us...,We have a beautiful landscaped garden with tab...,We live on the premises so we are always on ha...,Welcome to our cottage and we hope you will en...,NaN,NaN,https://a0.muscache.com/im/pictures/525706/050...,NaN,193005,https://www.airbnb.com/users/show/193005,Teresa,2010-08-06,Dublin Ireland,"I am a friendly outgoing Irishwoman ,who love...",within an hour,100%,NaN,t,https://a0.muscache.com/im/users/193005/profil...,https://a0.muscache.com/im/users/19

In [87]:

data_extended['host_neighbourhood'].value_counts(dropna=False)

NaN                                    3158
North City Central/O'Connell Street     497
Docklands                               455
Phibsborough                            402
The Liberties                           391
Ranelagh and Rathmines                  284
Ballsbridge                             258
Drumcondra                              244
Old City                                241
Stoneybatter/Arbour Hill                221
Trinity College                         207
Smithfield                              206
Georgian Quarter                        196
Donnybrook                              165
Grafton Street/St. Stephens Green       160
Portobello                              155
Rathgar                                 150
Kilmainham                              142
Temple Bar                              134
Cabra                                    92
Sandymount                               91
East Wall                                84
Clontarf                        

In [ ]:
#WHAT ARE THE MOST EXPENSIVE AREAS OF THE CITY TO RENT ON AIRBNB IN DUBLIN?

#The aim here is to, for each neighbourhood, break down the rentals into private rooms, single bed apartments..
#and 2 bed apartments. For ech of the 3 categories get an average. The average of these 3 averages will represent..
#the average cost of a rental for that area. 


#Which part of town gets the best reviews for location? for transport?

In [113]:
#create a copy of the data_extended data.
data_areas = data_extended.copy()


In [132]:
def get_list_of_uncommon_neighbourhoods(subset, min_freq):
    """ takes a subset of the dataset
    returns a list of the beighbourhoods that should be removed 
    """
    v_counts = subset['host_neighbourhood'].value_counts()
    uncommon_list = list(v_counts[v_counts < min_freq].index)    
    return uncommon_list

In [152]:
#Clean up the data
#1. Remove rows that have NaN for neighbourhood
#2. Remove data from shared rooms as they are fairly infrequent and are likely to have quite different prices
#3. Remove house with 3 or more bedrooms as we will focus on comparing 1 and 2 bedroom rentals (As they are more frequent)
#4. Remove data from neighbourhoods with only a small number of datapoints. We want at least %min_freq% houses and 
#at least %min_freq% private rooms for an area to qualify to be kept.

#1. Remove rows that have NaN for neighbourhood
data_areas.dropna(subset=['host_neighbourhood'], inplace=True)

#2. Remove data from shared rooms
data_areas = data_areas[~(data_areas['room_type'] == 'Shared room')]

#3 Remove houses with 3 or more bedrooms
data_areas = data_areas[~(data_areas['bedrooms'] > 2.0)]

#4. Remove data from neighbourhoods with only a small number of datapoints.

#The min number of times the neighbourhood needs to feature in the list to be considered for comparision 
min_freq = 5

# Get list of neigbourhoods that dont have enough samples for: entire 1 bed home/apt
subset_entirehomes_1b = data_areas[(data_areas['room_type'] =='Entire home/apt') & (data_areas['bedrooms'] ==1.0)]
ignore_hoods1_1bed = get_list_of_uncommon_neighbourhoods(subset_entirehomes_1b, min_freq)
print(ignore_hoods1_1bed)

# Get list of neigbourhoods that dont have enough samples for: entire 2 bed home/apt
subset_entirehomes_2b = data_areas[(data_areas['room_type'] =='Entire home/apt') & (data_areas['bedrooms'] ==2.0)]
ignore_hoods1_2bed = get_list_of_uncommon_neighbourhoods(subset_entirehomes_2b, min_freq)
print(ignore_hoods1_2bed)

# Get list of neigbourhoods that dont have enough samples for: private room
subset_privaterooms = data_areas[data_areas['room_type'] =='Private room']
ignore_hoods2 = get_list_of_uncommon_neighbourhoods(subset_privaterooms, min_freq)
print(ignore_hoods2)

ignore_hoods = list(set().union(ignore_hoods1_1bed,ignore_hoods1_2bed, ignore_hoods2 ))
print(ignore_hoods)

#get rid of the datapoints that are in the neigbourhoods to be ignored
data_areas_clean = data_areas[~data_areas['host_neighbourhood'].isin(ignore_hoods)].copy()


['Addison Park', 'Kimmage', 'Islandbridge', 'Isle of Dogs', 'Ballymun', 'Raheny', 'Churchtown', 'Fairview', 'Dartry', 'Santry', 'Glasnevin', 'Ballyfermot', 'Bonnybrook/Darndale', 'De Pijp', 'Downtown Montreal', 'Finglas', 'West Village', 'Rathfarnham', 'Fulham', 'Mount Merrion', 'Clondalkin', 'Terézváros - District VI.', 'Templeogue', 'Terenure', 'Ardilea']
['Kimmage', 'Beaumont', 'Churchtown', 'Glasnevin', 'Terézváros - District VI.', 'Ballyfermot', 'Artane', 'Pelletstown', "North City Central/O'Connel Street", 'Fairview', 'Rathfarnham', 'Marino', 'Donnycarney', 'Dublin Industrial Estate', 'Addison Park', 'Ixelles', 'Richmond District', 'Islandbridge', 'Oud-West', 'Knocklyon', 'Tour Eiffel - Champ de Mars', 'Finglas', 'De Pijp']
['Islandbridge', 'Donnycarney', "North City Central/O'Connel Street", 'Pelletstown', 'Terézváros - District VI.', 'Dublin Industrial Estate', 'Woluwe-Saint-Lambert', 'Kilmore', 'Auteuil', 'Rose Bay/Dover Heights', 'Edenmore', 'LB of Haringey', 'Opéra - Grands 

In [194]:
#Clean up the price column
#1. Get rid of dollar signs and commas
#2. Convert to floats
#3. Investigate outliers

#1. Get rid of dollar signs and commas
#2. Convert to floats
data_areas_clean['price'] = data_areas_clean['price'].replace('[^.0-9]', '', regex=True).astype(float)

#3. Investigate outliers
#Looking at outliers greater than 1000 dollars per night: There are 6 of these. I get rid of all six because..
#Two have broken links. Can't guarantee their integrity
#Three were found to be erroneous prices somehow. 
#One was a correct listings (very fancy rental). However this will skew results in its neighbourhood as it's..
#price is significantly than anything else in Dublin.
#??I should really dig into some of the rentals in the 500 to 1000 dollar branket to investigate.
data_areas_clean = data_areas_clean[~(data_areas_clean['price'] > 1000)]




In [195]:
#Divide the data into Private and Entire house/apt (1 and 2 bedroom) dataframes and sort into groups

data_privateroom = data_areas_clean[data_areas_clean['room_type'] == 'Private room']
privateroom_groups = data_privateroom['price'].groupby(data_areas_clean['host_neighbourhood'])

data_house_1bed = data_areas_clean[(data_areas_clean['room_type'] == 'Entire home/apt') & (data_areas_clean['bedrooms'] == 1.0)]
house_1bed_groups = data_house_1bed['price'].groupby([data_areas_clean['host_neighbourhood']])

data_house_2bed = data_areas_clean[(data_areas_clean['room_type'] == 'Entire home/apt') & (data_areas_clean['bedrooms'] == 2.0)]
house_2bed_groups = data_house_2bed['price'].groupby([data_areas_clean['host_neighbourhood']])



In [196]:
data_privateroom['price'].head(3)

7     60.0
8     55.0
10    55.0
Name: price, dtype: float64

In [213]:
#Combine the averages for the 3 lists

averages = (house_1bed_groups.mean() + house_2bed_groups.mean() + privateroom_groups.mean())/3

#Remove some neighbourhoods that crept in on 2 or fewer lists and therefore have no average across the three

averages = averages.dropna()

averages.sort_values(ascending=False)

host_neighbourhood
Ballsbridge                            138.434360
Docklands                              137.522308
Trinity College                        134.457321
Sandymount                             131.360534
Phibsborough                           126.067187
Georgian Quarter                       124.525343
Grafton Street/St. Stephens Green      123.665263
Drumcondra                             122.935871
Old City                               122.891443
North City Central/O'Connell Street    121.766063
Portobello                             119.925952
Temple Bar                             119.403360
Smithfield                             118.910385
The Liberties                          118.064577
East Wall                              117.465359
Ranelagh and Rathmines                 111.835897
Rathgar                                110.921225
Ringsend/Irishtown                     108.407738
North Strand                           106.368081
Stoneybatter/Arbour Hill       

In [ ]:
#Question: WHICH AREAS GET THE BEST AVERAGE LOCATION SCORE

In [214]:
#create a copy of the data_extended data.
data_popular = data_extended.copy()

In [217]:
data_popular.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
1,44077,https://www.airbnb.com/rooms/44077,20190122134607,2019-01-22,cosy comfortable Irish cottage twin,Our house was built in 1937 when there was ple...,"Hi, My name is Teresa and we have 3 rooms ava...",Our house was built in 1937 when there was ple...,none,I like our neighbourhood as there is no shorta...,Dundrum was always known as a Spa Town and the...,There is a bus stop 2 mins up the road from us...,We have a beautiful landscaped garden with tab...,We live on the premises so we are always on ha...,Welcome to our cottage and we hope you will en...,NaN,NaN,https://a0.muscache.com/im/pictures/525706/050...,NaN,193005,https://www.airbnb.com/users/show/193005,Teresa,2010-08-06,Dublin Ireland,"I am a friendly outgoing Irishwoman ,who love...",within an hour,100%,NaN,t,https://a0.muscache.com/im/users/193005/profil...,https://a0.muscache.com/im/users/193005/profil...,Churchtown,3,3,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,"Churchtown, Ireland",Churchtown,Dn Laoghaire-Rathdown,NaN,Churchtown,NaN,Dublin 14,Dublin,"Churchtown, Ireland",IE,Ireland,53.28995,-6.258126,t,Bed and breakfast,Private room,2,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,Kitchen,""Free parking on premis...",NaN,$50.00,NaN,NaN,NaN,$10.00,1,$0.00,3,60,3,3,60,60,3.0,60.0,a week ago,t,13,25,55,309,2019-01-22,182,47,2011-03-20,2018-12-09,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,NaN,t,f,moderate,t,t,3,0,3,0,1.91


In [219]:
#1. Remove rows that have NaN for neighbourhood
data_popular.dropna(subset=['host_neighbourhood'], inplace=True)

#1. Remove rows that have NaN for Location Rating
data_popular.dropna(subset=['review_scores_location'], inplace=True)

In [220]:
data_popular['review_scores_location'].value_counts(dropna=False)

10.0    2902
9.0     1992
8.0      347
7.0       37
6.0       28
4.0        9
2.0        8
5.0        3
Name: review_scores_location, dtype: int64

In [221]:
#Get rid of neighbourhoods that dont have enough samples

#The min number of times the neighbourhood needs to feature in the list to be considered for comparision 
min_freq = 20

# Get list of neigbourhoods that dont have enough samples for: entire 1 bed home/apt
ignore_these_areas = get_list_of_uncommon_neighbourhoods(data_popular, min_freq)
print(ignore_these_areas)

#get rid of the datapoints that are in the neigbourhoods to be ignored
data_pop_clean = data_popular[~data_popular['host_neighbourhood'].isin(ignore_these_areas)].copy()

['Bonnybrook/Darndale', 'Finglas', 'Dartry', 'Mount Merrion', 'Addison Park', 'Churchtown', 'Ardilea', 'Marino', 'Knocklyon', 'Ballymun', 'Courtlands', "North City Central/O'Connel Street", 'Templeogue', 'Ballinteer', 'Islandbridge', 'Terézváros - District VI.', 'Riverston Abbey', 'Dublin Industrial Estate', 'Pelletstown', 'Donnycarney', 'Grace Park', 'De Pijp', 'Ixelles', 'West Village', 'Saint-Germain-des-Prés - Odéon', 'Passy', 'Fulham', 'Downtown Montreal', 'Richmond District', 'Woluwe-Saint-Lambert', 'Opéra - Grands Boulevards', 'MLK & 183', 'Tour Eiffel - Champ de Mars', 'Auteuil', 'Edenmore']


In [225]:
rating_groups = data_pop_clean['review_scores_location'].groupby(data_pop_clean['host_neighbourhood'])

In [231]:
rating_groups.mean().sort_values(ascending=False)

host_neighbourhood
Temple Bar                             9.889831
Grafton Street/St. Stephens Green      9.835616
Trinity College                        9.754011
Georgian Quarter                       9.752874
Ballsbridge                            9.709360
Smithfield                             9.696809
Docklands                              9.685864
Old City                               9.678899
Sandymount                             9.666667
Portobello                             9.632353
Dundrum                                9.616667
North City Central/O'Connell Street    9.598624
Ranelagh and Rathmines                 9.584034
Santry                                 9.583333
Booterstown                            9.558824
Rathgar                                9.530973
Clontarf                               9.507692
Monkstown                              9.500000
Raheny                                 9.480000
Blackrock                              9.411765
Ringsend/Irishtown   